In [ ]:
import pymed as pm
import json

In [ ]:
#Create Query function
def get_query_from_keywords(keywords):
    query = ''
    for keyword_set in keywords:
        str_aux =''
        for single_key in keyword_set:
            if single_key==keyword_set[0]:
                str_aux += f'({single_key}[Title/Abstract] OR '
            elif single_key==keyword_set[-1]:
                str_aux += f'{single_key}[Title/Abstract] OR '
            else:
                str_aux += f'{single_key}[Title/Abstract] OR '
            
            if single_key==keyword_set[0]:
                str_aux += f'{single_key}[MeSH Major Topic] OR '
            elif single_key==keyword_set[-1]:
                str_aux += f'{single_key}[MeSH Major Topic])'
            else:
                str_aux += f'{single_key}[MeSH Major Topic] OR '
        
        if keyword_set==keywords[-1]:
            query += str_aux 
        else:
            query += str_aux + ' AND '
    return query

In [ ]:
#Query words
Biomedical =  ["Anesthesiology", "Biomedical", "Biomedicine", "Bioinformatics", "Cardiovascular", "Clinical", "Critical Care", "Dermatology", "Disease", "Drug", 
                "Gastroenterology", "Genetics", "Genomics", "Gynecology", "Health", "Healthcare", "Hematology", "Life sciences", "Medical", "Medicine", "Nephrology", 
                "Neurology", "Obstetrics", "Oncology", "Ophthalmology", "Orthopedic", "Pathology", "Pediatrics", "Pharmacology",  "Protein", "Psychiatry", "Radiology", 
                "Surgery", "Urology", "Vaccine", "Immunology"]

AI =  ["AI", "Artificial Intelligence", "Deep Learning", "Machine Learning"]

XAI =  ["Black Box Models", "Explainable", "Inerpretability", "Explainability", "Opacity", "Opaque", "xAI"]

Causal =  ["Causal", "Causation", "Counterfactual"]

#### Biomed+Ai+xAI query

In [ ]:
query = [Biomedical,AI, XAI]
query_str = get_query_from_keywords(query)

In [ ]:
PUBMED = pm.PubMed(tool="MyTool", email='<USER EMAIL>')
raw = PUBMED.query(query_period, max_results=9999)
fulldump=list(raw)

In [ ]:
#Create list containing paper information
full_list_file= []
for element in fulldump:
    if element.abstract is not None and element.title is not None:
        pass_criteria = [False, False, False] 
        for biomedicalkw in Biomedical:
            if biomedicalkw.lower() in element.abstract.lower() or biomedicalkw in element.title.lower():
                pass_criteria[0]=True
                break
        for aikw in AI:
            if aikw.lower() in element.abstract.lower() or aikw in element.title.lower():
                pass_criteria[1]=True
                break
        for xmlkw in XAI:
            if xmlkw.lower() in element.abstract.lower() or xmlkw in element.title.lower():
                pass_criteria[2]=True
                break

        if pass_criteria == [True, True, True]:
            aut_list=''
            for author in element.toDict()['authors']:
                try:
                    aut_list += author['lastname'] + ' ' + author['firstname'] +', '
                except:
                    aut_list+= 'None, '
            aut_list=aut_list[:-2]
            dict_aux={}
            try:
                dict_aux['title']=element.title
            except:
                dict_aux['title']='None'
            try:
                dict_aux['abstract']=element.abstract
            except:
                dict_aux['abstract']='None'
            try:
                dict_aux['doi']=element.doi
            except:
                dict_aux['doi']='None'
            try:
                dict_aux['journal']=element.journal
            except:
                dict_aux['journal']='None'
            try:
                dict_aux['date']=f'{element.publication_date.year}-{element.publication_date.month}-{element.publication_date.day}'
            except:
                dict_aux['date']='None'
            try:
                dict_aux['keywords']=element.keywords
            except:
                dict_aux['keywords']='None'
            dict_aux['authors']=aut_list
            full_list_file.append(dict_aux)

dump_filename = 'papers_biomed_ai_xai.json'
with open(dump_filename, 'w', encoding='utf-8') as f:
    json.dump(full_list_file, f)


In [ ]:
#extract keywords from abstract and title

with open('papers_biomed_ai_xai.json','r') as fullfile:
    full_list_file=json.load(fullfile)

keyword_list_biomed_ai_xai= []

id = 0
for paper in full_list_file:
    keyword_list = []
    if paper['abstract'] is not None and paper['title'] is not None:
        for biomedicalkw in Biomedical:
            if biomedicalkw in paper['abstract'] or biomedicalkw in paper['title'] or biomedicalkw in paper['keywords'] :
                keyword_list.append(biomedicalkw)
        for aikw in AI:
            if aikw in paper['abstract'] or aikw in paper['title'] or aikw in paper['keywords']:
                keyword_list.append(aikw)
        for xmlkw in XAI:
            if xmlkw in paper['abstract'] or xmlkw in paper['title'] or xmlkw in paper['keywords']:
                keyword_list.append('Explainability')

        keyword_dict = {}

        keyword_dict['Index'] = id
        keyword_dict['keywords'] = keyword_list

        keyword_list_biomed_ai_xai.append(keyword_dict)

    id += 1

#write it on a json file
dump_filename = 'keyword_biomed_ai_xai.json'

with open(dump_filename, 'w', encoding='utf-8') as f:
    json.dump(keyword_list_biomed_ai_xai, f)

#### Biomed+Ai+xAI+Causal query

In [ ]:
query = [Biomedical,AI, XAI, Causal]
query_str = get_query_from_keywords(query)

In [ ]:
PUBMED = pm.PubMed(tool="MyTool", email='<USER EMAIL>')
raw = PUBMED.query(query_period, max_results=9999)
fulldump=list(raw)

In [ ]:
#Create list containing paper information
full_list_file= []
for element in fulldump:
    if element.abstract is not None and element.title is not None:
        pass_criteria = [False, False, False, False]
        for biomedicalkw in Biomedical:
            if biomedicalkw.lower() in element.abstract.lower() or biomedicalkw in element.title.lower():
                pass_criteria[0]=True
                break
        for aikw in AI:
            if aikw.lower() in element.abstract.lower() or aikw in element.title.lower():
                pass_criteria[1]=True
                break
        for xmlkw in XAI:
            if xmlkw.lower() in element.abstract.lower() or xmlkw in element.title.lower():
                pass_criteria[2]=True
                break
        for causalkw in Causal:
            if causalkw.lower() in element.abstract.lower() or causalkw in element.title.lower():
                pass_criteria[3]=True
                break
        if pass_criteria == [True, True, True, True]:
            aut_list=''
            for author in element.toDict()['authors']:
                try:
                    aut_list += author['lastname'] + ' ' + author['firstname'] +', '
                except:
                    aut_list+= 'None, '
            aut_list=aut_list[:-2]
            dict_aux={}
            try:
                dict_aux['title']=element.title
            except:
                dict_aux['title']='None'
            try:
                dict_aux['abstract']=element.abstract
            except:
                dict_aux['abstract']='None'
            try:
                dict_aux['doi']=element.doi
            except:
                dict_aux['doi']='None'
            try:
                dict_aux['journal']=element.journal
            except:
                dict_aux['journal']='None'
            try:
                dict_aux['date']=f'{element.publication_date.year}-{element.publication_date.month}-{element.publication_date.day}'
            except:
                dict_aux['date']='None'
            try:
                dict_aux['keywords']=element.keywords
            except:
                dict_aux['keywords']='None'
            dict_aux['authors']=aut_list
            full_list_file.append(dict_aux)

dump_filename = 'papers_biomed_ai_xai_causal.json'
with open(dump_filename, 'w', encoding='utf-8') as f:
    json.dump(full_list_file, f)

In [ ]:
#extract keywords from abstract and title

with open('JSON_files/papers_biomed_ai_xai_causal.json','r') as fullfile:
    full_list_file=json.load(fullfile)

keyword_list_biomed_ai_xai_causal= []

id = 0
for paper in full_list_file:
    keyword_list = []
    if paper['abstract'] is not None and paper['title'] is not None:
        for biomedicalkw in Biomedical:
            if biomedicalkw in paper['abstract'] or biomedicalkw in paper['title'] or biomedicalkw in paper['keywords'] :
                keyword_list.append(biomedicalkw)
        for aikw in AI:
            if aikw in paper['abstract'] or aikw in paper['title'] or aikw in paper['keywords']:
                keyword_list.append(aikw)
        for xmlkw in XAI:
            if xmlkw in paper['abstract'] or xmlkw in paper['title'] or xmlkw in paper['keywords']:
                keyword_list.append('Explainability')
        for causalkw in Causal:
            if causalkw in paper['abstract'] or causalkw in paper['title'] or causalkw in paper['keywords']:
                keyword_list.append('Causal')

        keyword_dict = {}

        keyword_dict['Index'] = id
        keyword_dict['keywords'] = keyword_list

        keyword_list_biomed_ai_xai_causal.append(keyword_dict)

    id += 1

#write it on a json file
dump_filename = 'keyword_biomed_ai_xai_causal.json'

with open(dump_filename, 'w', encoding='utf-8') as f:
    json.dump(keyword_list_biomed_ai_xai_causal, f)